In [18]:
import pymongo
from bson.json_util import dumps
uri = "mongodb://localhost:27017"
client = pymongo.MongoClient(uri)
db = client.dionaeareference

In [19]:
db.connections

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'dionaeareference'), 'connections')

In [31]:
cari = db.connections.find({})

In [32]:
list(cari)

['_id',
 'connection',
 'connection_type',
 'connection_transport',
 'connection_protocol',
 'connection_timestamp',
 'connection_root',
 'connection_parent',
 'local_host',
 'local_port',
 'remote_host',
 'remote_hostname',
 'remote_port']

In [46]:
pipeline = [
    {
        '$match': {}
    }, {
        '$lookup': {
            'from': 'logins', 
            'localField': 'connection', 
            'foreignField': 'connection', 
            'as': 'login'
        }
    }, {
        '$unwind': '$login'
    }, {
        '$lookup': {
            'from': 'mysql_commands', 
            'localField': 'connection', 
            'foreignField': 'connection', 
            'as': 'mysql_command'
        }
    }, {
        '$unwind': '$mysql_command'
    }, {
        '$lookup': {
            'from': 'mysql_command_ops', 
            'localField': 'mysql_command.mysql_command_cmd', 
            'foreignField': 'mysql_command_cmd', 
            'as': 'mysql_command_op'
        }
    }
]
cari = db.connections.aggregate(pipeline).next()
from pprint import pprint
pprint(cari)

{'_id': ObjectId('5d1d540605014d6d8686f140'),
 'connection': 41,
 'connection_parent': None,
 'connection_protocol': 'mysqld',
 'connection_root': 41,
 'connection_timestamp': 1468416870,
 'connection_transport': 'tcp',
 'connection_type': 'accept',
 'local_host': '202.43.92.50',
 'local_port': 3306,
 'login': {'_id': ObjectId('5d1d544d05014d6d8686f3ce'),
           'connection': 41,
           'login': 8,
           'login_password': '',
           'login_username': 'root'},
 'mysql_command': {'_id': ObjectId('5d028a21c310f00592b87679'),
                   'connection': 41,
                   'mysql_command': 1,
                   'mysql_command_cmd': 14},
 'mysql_command_op': [{'_id': ObjectId('5ce659e2f5a8e73894dedf7a'),
                       'mysql_command_cmd': 14,
                       'mysql_command_op': 15,
                       'mysql_command_op_name': 'COM_PING'}],
 'remote_host': '124.172.136.143',
 'remote_hostname': '',
 'remote_port': 3655}
